In [12]:
import geopandas as gpd
import os

# WFS type name
typename = "erhaltungsverordnungsgebiete:erhaltgeb_es"

# Downloading GeoDataFrame directly
gdf = gpd.read_file(
    f"{url}?SERVICE=WFS&VERSION=2.0.0&REQUEST=GetFeature&TYPENAMES={typename}&OUTPUTFORMAT=application/json"
)

# Creating a directory
os.makedirs("sources", exist_ok=True)

# Saving as GeoJSON
gdf.to_file("sources/milieuschutzgebiete.geojson", driver="GeoJSON")

# Viewing the first lines
print(gdf.head())

                    id schluessel bezirk  \
0  erhaltgeb_es.ES0101     ES0101  Mitte   
1  erhaltgeb_es.ES0102     ES0102  Mitte   
2  erhaltgeb_es.ES0103     ES0103  Mitte   
3  erhaltgeb_es.ES0104     ES0104  Mitte   
4  erhaltgeb_es.ES0105     ES0105  Mitte   

                                         gebietsname  f_gvbl_dat  f_in_kraft  \
0                                        Poststadion  30.12.1988  31.12.1988   
1                                 Spandauer Vorstadt  25.06.1993  26.06.1993   
2  Südliche Brunnenstraße Teile der Rosenthaler V...  09.12.1995  10.12.1995   
3                            Friedrich-Wilhelm-Stadt  31.08.1996  01.09.1996   
4                     Dorotheenstadt, Friedrichstadt  10.04.1997  11.04.1997   

  ae_gvbldat ae_inkraft fl_in_ha  \
0       None       None     53.2   
1       None       None    109.1   
2       None       None     18.8   
3       None       None     69.5   
4       None       None     98.7   

                                     

In [13]:
gdf = gdf.rename(columns={
    "schluessel": "code",
    "bezirk": "district",
    "gebietsname": "zone_name",
    "f_gvbl_dat": "publication_date",
    "f_in_kraft": "effective_date",
    "ae_gvbldat": "alt_publication_date",
    "ae_inkraft": "alt_effective_date",
    "fl_in_ha": "area_ha"
})

In [14]:
# Remove geometry (otherwise Excel will not be able to save MULTIPOLYGON correctly)
df = gdf.drop(columns='geometry')

# Save to Excel file
df.to_excel("sources/milieuschutzgebiete.xlsx", index=False)

# Save to CSV
df.to_csv("sources/milieuschutz_areas.csv", index=False)

In [15]:
# Copy the GeoDataFrame so as not to alter the original
gdf_wkt = gdf.copy()

# Convert geometry to string (WKT)
gdf_wkt["geometry"] = gdf_wkt["geometry"].apply(lambda geom: geom.wkt)

# Save to CSV with geometry as text
gdf_wkt.to_csv("sources/milieuschutz_areas_with_geom.csv", index=False)

/tmp/ipython-input-15-393268848.py:5: UserWarning: Geometry column does not contain geometry.
  gdf_wkt["geometry"] = gdf_wkt["geometry"].apply(lambda geom: geom.wkt)


In [18]:
# And if we want to make a GeoDataFrame from this CSV again:
import pandas as pd
from shapely import wkt

df = pd.read_csv("sources/milieuschutz_areas_with_geom.csv")
df["geometry"] = df["geometry"].apply(wkt.loads)
gdf_restored = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:25833")